<a href="https://colab.research.google.com/github/MKJJang/airflow/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LSTM + Transformer 모델 구조**

**(1) 입력 데이터**
X: 광고 노출 수, 클릭률(CTR), 전환율(CVR), 할인율, 계절성 지수
y: 일별/주별/월별 매출

**(2) 모델 아키텍처**
LSTM Layer: 단기적인 패턴 학습
Transformer Encoder Layer: Self-Attention을 활용하여 장기 패턴 학습
Fully Connected Layer: 예측값 출력 (최종 매출, 광고 성과 예측)

In [ ]:
import torch
import torch.nn as nn

class LSTM_Transformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, seq_length):
        super(LSTM_Transformer, self).__init__()

        # LSTM Layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        # Fully Connected Output Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # LSTM Forward Pass
        transformer_out = self.transformer(lstm_out)  # Transformer Forward Pass
        output = self.fc(transformer_out[:, -1, :])  # Fully Connected Layer
        return output

# 모델 초기화
model = LSTM_Transformer(input_dim=10, hidden_dim=64, num_layers=2, output_dim=1, seq_length=30)

**LSTM + Transformer 모델 평가 및 튜닝**

**평가 지표**
MAE (Mean Absolute Error): 절대 오차

RMSE (Root Mean Squared Error): 제곱 평균 오차

MAPE (Mean Absolute Percentage Error): 백분율 오차

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    return mae, rmse

# 실제 값 vs 예측 값 평가
mae, rmse = evaluate(y_test, y_pred)
print(f"MAE: {mae}, RMSE: {rmse}")

**하이퍼파라미터 튜닝**

*   LSTM: Hidden Size, Number of Layers, Dropout
*   Transformer: Number of Heads, Attention Layers, Feedforward Size
*   Learning Rate: 0.001 ~ 0.0001 튜닝 (Adam Optimizer)

**6. 실제 적용 사례**

📌 Case 1: 이커머스 광고 성과 예측

목표: 광고 예산 대비 ROAS (Return on Ad Spend) 최적화

*   목표: 광고 예산 대비 ROAS (Return on Ad Spend) 최적화
*   모델 적용: LSTM + Transformer로 광고 예산과 CTR, CVR 학습
*   성과: 기존 XGBoost 모델 대비 광고 예측 정확도 18% 향상

📌 Case 2: 프로모션 매출 예측

*   목표: 특정 할인 프로모션이 매출에 미치는 영향 분석
*   모델 적용: Transformer로 장기적인 프로모션 효과 분석
*   성과: 기존 LSTM 모델 대비 장기 매출 예측 정확도 22% 향상

**향후 발전 방향**

*   Reinforcement Learning 결합 → 광고 예산 자동 최적화
*   Graph Neural Network (GNN) 적용 → 고객 네트워크 분석 통한 예측 개선
*   AutoML 활용 → 최적의 모델 자동 탐색

이커머스 매출 & 광고 최적화에 LSTM +Transformer 모델을 활용하면 더 높은 예측 정확도와 비즈니스 성과를 달성할 수 있다.

1. LSTM 학습셋 & 테스트셋 split

(1) 일반적인 머신러닝과의 차이

일반적인 머신러닝에서는 데이터를 랜덤하게 섞어서 훈련(train), 검증(validation), 테스트(test) 데이터로 분리한다.

하지만 LSTM은 시계열 데이터를 다루므로, 시간 순서를 유지하면서 데이터를 분리해야 합니다.

(2) 데이터 분할 예시

예제: 지난 3년간의 광고 및 매출 데이터를 학습하고, 최근 6개월 데이터를 테스트

램덤 분할이 아닌 "시간순으로" 과거 데이터를 학습하고 미래 데이터를 평가함

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 샘플 데이터 생성
data = pd.DataFrame({
    'date': pd.date_range(start='2020-01-01', periods=1000, freq='D'),  # 1000일간의 데이터
    'ad_spend': np.random.randint(100, 500, size=1000),  # 광고비
    'clicks': np.random.randint(1000, 5000, size=1000),  # 클릭 수
    'sales': np.random.randint(50, 500, size=1000)  # 매출량 (타겟 변수)
})

# 날짜 기준 정렬 (시계열 데이터이므로 랜덤 섞지 않음)
data = data.sort_values(by='date')

# 훈련 데이터(80%)와 테스트 데이터(20%) 분리
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# 출력
print(train_size)
print(f"Train Data: {len(train_data)} rows")
print(f"Test Data: {len(test_data)} rows")

800
Train Data: 800 rows
Test Data: 200 rows


In [5]:
data.head()

,date,ad_spend,clicks,sales
0,2020-01-01,291,1931,150
1,2020-01-02,200,2429,219
2,2020-01-03,311,2084,156
3,2020-01-04,446,2463,105
4,2020-01-05,157,1319,460


**2. LSTM 학습과 테스트셋 적용**

LSTM 모델에 데이터를 입력하기 위해선 시계열 윈도우(Windowing) 방식을 사용해야 함.

**(1) LSTM을 위한 데이터 변환**

✅ 시계열 데이터는 X와 y를 다음과 같이 변환해야 함


*   X: seq_length 만큼의 과거 데이터를 입력값으로 사용
*   y: 예측할 미래 데이터(타겟)

In [8]:
import torch
from sklearn.preprocessing import MinMaxScaler

# 스케일링 (LSTM은 수치 안정성이 중요하기 때문에 정규화 필요)
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_data[['ad_spend', 'clicks', 'sales']])
scaled_test = scaler.transform(test_data[['ad_spend', 'clicks', 'sales']])

# 시퀀스 데이터 변환 함수
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # 과거 seq_length 만큼을 입력
        y.append(data[i+seq_length, -1])  # 마지막 컬럼(매출) 예측
    return np.array(X), np.array(y)

# 훈련 데이터셋 변환
seq_length = 30  # 30일씩 묶어서 예측
X_train, y_train = create_sequences(scaled_train, seq_length)
X_test, y_test = create_sequences(scaled_test, seq_length)

# PyTorch 텐서 변환
# LSTM을 포함한 PyTorch의 모든 신경망 모델은 텐서(Tensor)형태의 입력을 필요로 한다.
# 따라서 데이터를 NumPy배열 -> PyTorch 텐서로 변환해야 한다.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

print(f"X_train shape: {X_train_tensor.shape}")  # (샘플 수, 시퀀스 길이, 특성 수)
print(f"y_train shape: {y_train_tensor.shape}")  # (샘플 수,)

# 각 샘플마다 30일의 과거 데이터를 보고 다음 날을 예측하는 방식
# seq_length = 30
# i : seq_length

X_train shape: torch.Size([770, 30, 3])
y_train shape: torch.Size([770])


**텐서(Tensor)란?**

Tensor는 다차원 배열(행렬)을 일반화한 데이터 구조로, PyTorch에서 딥러닝 모델을 학습하는 데 사용된다.

쉽게 말해 NumPy 배열과 유사하지만, GPU 연산이 가능한 데이터 구조입니다.

*   스칼라 (Scalar)	단일 값 (예: 5)
*   벡터 (Vector)	1차원 배열 (예: [1, 2, 3])
*   행렬 (Matrix)	2차원 배열 (예: [[1, 2], [3, 4]])
*   텐서 (Tensor)	3차원 이상 (예: [[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

PyTorch의 모든 신경망 연산은 텐서 연산을 기반으로 동작한다.

**3. LSTM 모델 학습 및 평가**

In [9]:
import torch.nn as nn
#신경망 모듈로 Neural Network를 쉽게 구축할 수 있도록 다양한 클래스와 함수를 제공하는 PyTorch의 핵심모듈. 다양한 층(layer), 활성화함수(activation function), 손실함수(loss function), 정규화(normalization)등을 효율적으로 정의할 수 있다.
import torch.optim as optim
#최적화 알고리즘을 (optimizer)를 사용하기 위한 모듈을 불러오는 코드. 신경망 학습 과정에서 가중치(weight)를 업데이트 하는 다양한 최적화 알고리즘을 제공.
#최적화 알고리즘은 손실함수(loss function)의 값을 최소화 하기 위해 신경망의 가중치(weight)와 편향(bias)을 조정하는 역할을 한다.

class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM_Model, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        final_output = lstm_out[:, -1, :]  # 마지막 타임스텝의 hidden state 사용
        return self.fc(final_output)

# 모델 초기화
input_dim = 3  # 광고비, 클릭수, 매출
hidden_dim = 64
num_layers = 2
output_dim = 1
model = LSTM_Model(input_dim, hidden_dim, num_layers, output_dim)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 평가 (테스트셋 적용)
model.eval()
y_test_pred = model(X_test_tensor).detach().numpy()

# 성능 평가
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_test_pred)
print(f"Test MAE: {mae}")

Epoch 0, Loss: 0.3247513771057129
Epoch 10, Loss: 0.13537080585956573
Epoch 20, Loss: 0.08372555673122406
Epoch 30, Loss: 0.08716151118278503
Epoch 40, Loss: 0.08436091244220734
Test MAE: 0.27190238207590367


**optim.Adam**

optim.Adam은 Adaptive Moment Estimation(적응형 모멘트 추정) 의 약자로, SGD(Stochastic Gradient Descent)의 단점을 보완한 강력한 최적화 알고리즘입니다.
Adam은 모멘텀(Momentum)과 RMSProp(Root Mean Square Propagation) 을 결합하여, 학습률을 자동 조정하고 빠른 수렴을 유도합니다.

Adam은 학습률을 자동 조정하면서도, 각 파라미터마다 적응적으로 최적의 학습률을 설정하기 때문에 매우 효과적인 알고리즘입니다.

In [ ]:
# Adam Optimizer의 주요 하이퍼파라미터

optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,       # 학습률 (기본값 0.001)
    betas=(0.9, 0.999),  # 모멘텀 및 RMSProp 계수 (1차 모멘트와 2차 모멘트 계수)
    eps=1e-8,       # 작은 값 (Zero division 방지) (수치적 안정성을 위한 작은 값)
    weight_decay=0   # 가중치 감소 (L2 정규화) ((AdamW에서 더 적절하게 사용됨)
)

**Adam과 다른 Optimizer 비교**

Optimizer/	특징/	장점/	단점

SGD/	기본적인 경사 하강법/	단순하고 직관적	수렴/ 속도가 느림

Momentum/	SGD + 관성 효과/	지역 최소점에서 탈출 가능/	모멘텀 값 조정 필요

RMSProp/	학습률 자동 조정/	RNN에서 효과적/	적절한 alpha 조정 필요

Adam/	SGD + Momentum + RMSProp/	학습률 자동 조정, 빠른 수렴/	일부 문제에서 과적합 가능

AdamW/	Adam + Weight Decay/	과적합 방지/	Adam과 비슷하지만 일반화 성능 증가

In [ ]:
## 출력값 예시
## yaml (LSTM이 미래 매출을 예측할 수 있도록 학습됨)

# Epoch 0, Loss: 0.0456
# Epoch 10, Loss: 0.0152
# Epoch 20, Loss: 0.0083
# Test MAE: 3.12

# ✔ Epoch(학습 횟수)이 증가할수록 손실(Loss)이 감소 → 모델이 점점 좋아짐
# ✔ Test MAE 3.12 → 모델이 학습한 결과, 테스트 데이터에서 평균 3.12 정도의 오차가 발생
# ✔ 모델이 적절히 학습되었지만, MAE를 더 낮추기 위해 추가적인 하이퍼파라미터 튜닝이 가능함